# Plotting SFR Tracks of Galaxy Evolution

Used to plot the time evolution of the star-formation rate (SFR) for galaxies lying on the star-forming galaxy main sequence (SF GMS), or off by a certain number of dex – typically 1 dex.

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from datetime import datetime
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm, trange

import SiGMo as sgm

In [ ]:
project_dir = Path.cwd().parent
sfr79_dir = project_dir / 'data' / "SFR79_grids"
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.13.09" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3"
# snp_dir_GMSp1_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.55.31" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3_SFRoffset1.0"
# snp_dir_GMSm1_sim = project_dir / 'outputs' / '_tmp' / "2022.04.13-17.57.35" / "0_forward_from_z0.0729411607551602_1Gyr_dt1e-3_SFRoffset-1.0"

# z=2..0
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.30.50" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# z=1..0
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.39.39" / "0_forward_from_z1.0_to_z0_dt1e-3_SFRoffset0.0"

# z=2..0 with lower fgal=0.3
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.58.48" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"

# z=2..0 with higher fgal=0.5
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-20.09.21" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"


# on, above, below GMS, z=2...0

# on GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.04.19-19.30.50" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# 1 dex above GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.10.32" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset1.0"
# 1 dex below GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.20.25" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset-1.0"

# on GMS, from z=6...0
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.05-17.37.05" / "0_forward_from_z6.0_to_z0_dt1e-3_SFRoffset0.0"


# new initial gas masses (Tacconi+2020)
# on GMS
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.12-15.55.07" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# SFE=1, mgas set from SFE and SFR
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.12-19.32.33" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# SFE=1.5, mgas set from SFE and SFR
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.12-20.06.52" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"

# WAY LOWER fgal=0.1, SFE=1.5, mgas set from SFE and SFR
# snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.16-14.47.26" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"
# WAY LOWER fgal=0.1, SFE=1.5, mgas set from SFE and SFR
snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.05.16-14.59.46" / "0_forward_from_z2.0_to_z0_dt1e-3_SFRoffset0.0"




# Analysing BACKWARDS RUNS with scaled sMIR halo accretion rates

# factor 2
snp_dir_GMS_sim = project_dir / 'outputs' / '_tmp' / "2022.06.21-22.22.35" / "0_backward_2Gyr_dt1e-3_sMIR_scaling_basefactor2.0"




# ATTENTION! plotting directory is based on the (on-) GMS directory parent name/name here!
plot_dir = project_dir / 'plots' / '_tmp' / snp_dir_GMS_sim.parent.name / snp_dir_GMS_sim.name   # plot dir has now same datetime name as output dir
plot_dir.mkdir(exist_ok=True, parents=True)   # create plot dir only if necessary

### Read in

In [ ]:
single_snapshots = False

# how many objects per timestep?
n_envs = 1
# this is "unknown" unless looked up someplace else, but if 'None' will be determined from some Environment snapshot
n_halos = None
n_gals = None

# reading simulation results (SDSS)
env_grid_GMS_sim, halo_grid_GMS_sim, gal_grid_GMS_sim = sgm.read_all_snapshots_from_dir(snp_dir=snp_dir_GMS_sim,  # this is where you can plug in different offsets or on-GMS runs
                                                                                        n_envs=n_envs,
                                                                                        n_halos=n_halos,
                                                                                        n_gals=n_gals,
                                                                                        single_snapshots=single_snapshots)

In [ ]:
# re-create/populate halo_grid and galaxy_grid (for FIRST Environment only) if necessary
if not single_snapshots:
    for t, env_snp in enumerate(tqdm(env_grid_GMS_sim[0])):  # HARDCODED to 1st Environment only!
        for i_halo, halo in enumerate(env_snp.data['halos']):
            halo_grid_GMS_sim[i_halo, t] = sgm.Snapshot(halo)
            for i_gal, gal in enumerate(halo['galaxies']):
                gal_grid_GMS_sim[i_halo, t] = sgm.Snapshot(gal)  # HARDCODED to 1 Galaxy per Halo!!

### Plotting

In [ ]:
# define which data to plot
gal_grid = gal_grid_GMS_sim
halo_grid = halo_grid_GMS_sim
env_grid = env_grid_GMS_sim

# define what to plot
x_type = 'mstar'
x_type_name = r'log $M_*$'
x_type_unit = r'$M_\odot$'

# y_type = 'SFR'
# y_type_name = r'log $SFR$'
# y_type_unit = r'$M_\odot yr^{-1}$'

# y_type = 'macc'
# y_type_name = r'log $M_\mathrm{acc}$'
# y_type_unit = r'$M_\odot$'

# y_type = 'macc'
# y_type_name = r'log $\Delta M_\mathrm{acc}$'
# y_type_unit = r'$M_\odot$'

# y_type = 'SFE'
# y_type_name = r'$SFE$'
# y_type_unit = r'$\mathrm{Gyr}^{-1}$'

# y_type = 'SFE'
# y_type_name = r'$t_\mathrm{depl} = SFE^{-1}$'
# y_type_unit = r'$\mathrm{Gyr}$'

y_type = 'mgas'
y_type_name = r'log $M_\mathrm{gas}$'
y_type_unit = r'$M_\odot$'

c_type = 'mtot'
c_type_name = r'$M_{halo}$'
c_type_unit = r'$M_\odot$'
c_snp = 0
# c_snp = -1
c_alpha = 0.75
which_objects = range(0, len(gal_grid), 1)

# grab plotting data
x_data = []
y_data = []
c_data = []
label_l = []
for i in tqdm(which_objects):
    x_data.append([np.log10(gal.data[x_type]) for gal in gal_grid[i][:]])
    # x_data.append([np.log10(gal.data[x_type]) for gal in gal_grid[i][1:]])
    # y_data.append([np.log10(gal.data[y_type]) - 9. for gal in gal_grid[i][:]])
    y_data.append([np.log10(gal.data[y_type]) for gal in gal_grid[i][:]])
    # y_data.append([gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([1./gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([np.log10(gal_1.data[y_type] - gal_0.data[y_type]) for gal_0, gal_1 in zip(gal_grid[i][:-1], gal_grid[i][1:])])
    # y_data.append([halo.data[y_type] for halo in halo_grid[i][:]])
    # if gal_grid[i][c_snp].data['mgas'] == 0 and halo_grid[i][c_snp].data['mtot'] != 0:
    #     _c_data_i = [np.nan]
    # else:
    #     _c_data_i = [halo_grid[i][c_snp].data['mtot'] / gal_grid[i][c_snp].data['mgas']]
    c_data.append([halo_grid[i][c_snp].data[c_type]])
    # c_data.append(_c_data_i)
    label_l.append(r"$\mathrm{M}_\star$" + f"= {gal_grid[i, 0].data['mstar']:.2e} \t SFR = {gal_grid[i, 0].data['SFR'] / 10 ** 9:.2e}")


# normalise the colour data to make colour the data according to it (e.g. total halo mass mtot)
c_data_range = (np.nanmin(c_data), np.nanmax(c_data))
cmap = mpl.cm.viridis_r
norm = mpl.colors.LogNorm(vmin=c_data_range[0], vmax=c_data_range[1])
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


# grab all redshifts
z_list = [gal.data['z'] for gal in gal_grid[0][:]]


In [ ]:
# initialise plot
fig, ax = plt.subplots(figsize=(9.5, 6), constrained_layout=True)

# get redshift and lookbacktime min and max
z_start = env_grid[0, 0].data['z']
z_end = env_grid[0, -1].data['z']
lbt_start = env_grid[0, 0].data['lookbacktime']
lbt_end = env_grid[0, -1].data['lookbacktime']

# # plot GMS Leslie+2020
# xlin = np.linspace(x_data[0][0], x_data[-1][0], 500)
# # ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_start, log=True), ls='-.', marker='', color='xkcd:turquoise',
# #         # label=f"{lbt_start:.2f} Gyr ago (z={z_start:.1f})"
# #         )
# ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_end, log=True), ls=':', marker='', color='xkcd:turquoise',
#         # label=f"{lbt_end:.2f} Gyr ago (z={z_end:.1f})"
#         )


# additional DASHED LINES for highlighting time evolution

# plot lines at same redshift
redshifts_plot_l = [2., 1.5, 1., 0.5, 0.]
# redshifts_plot_l = np.append(np.linspace(1.99, 1.74, 6), 0.0001)
# redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
dl_data = redshifts_plot_l
dl_plot_l = []
dl_label_l = []
for redshift in redshifts_plot_l:
    # z_i = sgm.find_nearest_index(z_list, redshift)
    z_i = sgm.find_nearest_index(z_list[1:], redshift)
    z_line_x = [gal_x[z_i] for gal_x in x_data]
    z_line_y = [gal_y[z_i] for gal_y in y_data]
    dl_plot_l.append([z_line_x, z_line_y])
    dl_label_l.append(f"z={redshift:.2f}")
legend_title = "Lines of same redshift"


# # plot lines of same snapshots
# # snp_plot_l = list(range(1, 10, 1)) + list(range(10, 100, 10)) + list(range(100, 1000, 100))
# snp_plot_l = list(range(0, len(x_data[0]) + 1, 500))
# dl_data = snp_plot_l
# dl_plot_l = []
# dl_label_l = []
# for snp in snp_plot_l:
#     z_line_x = [gal_x[snp] for gal_x in x_data]
#     z_line_y = [gal_y[snp] for gal_y in y_data]
#     dl_plot_l.append([z_line_x, z_line_y])
#     dl_label_l.append(f"snap {snp}")
# legend_title = "Lines of same snapshots"


# dashed-lines colour map
dl_data_range = (np.nanmin(dl_data), np.nanmax(dl_data))
dl_cmap = mpl.cm.winter_r
dl_norm = mpl.colors.Normalize(vmin=dl_data_range[0], vmax=dl_data_range[1])
dl_mapper = mpl.cm.ScalarMappable(norm=dl_norm, cmap=dl_cmap)

for dl, dl_plot, dl_label in zip(dl_data, dl_plot_l, dl_label_l):
    ax.plot(dl_plot[0], dl_plot[1], ls='--', color=dl_mapper.to_rgba(dl), alpha=0.5, label=dl_label)


# plot actual values
for i, (x, y, c, label) in enumerate(zip(x_data, y_data, c_data, label_l)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax.plot(x, y, c=c_mapped, ls='-')
    # ax.plot(x[-1], y[-1], c=c_mapped, marker='o')


# add colour bar
fig.colorbar(mapper, ax=ax, label=f"{c_type_name} [{c_type_unit}] at z = {env_grid[0, c_snp].data['z']:.2f}")


# # plot one quantity like SFR79
# for i, c, label in zip(which_objects, c_data, label_l):
#     c_mapped = mapper.to_rgba(c)
#     c_mapped[:, -1] = c_alpha
#     ax[1].plot(c_data[i], SFR79_grid_xCGsim[i, -1], c=c_mapped, marker='o', label=label)


# additional fig and axes config
fig.suptitle(f'Tracing {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)


ax.set_xlabel(f"{x_type_name} [{x_type_unit}]")
ax.set_ylabel(f"{y_type_name} [{y_type_unit}]")


# this is not good, only necessary b/c of exceptionally high values
ax.set_xlim(4.5, 16)
# ax.set_ylim(3.8, 12.3)

# ax.set_yscale('log')



# ax.legend(title="Leslie+2020 Galaxy Main Sequence:")
# ax.legend(title="Lines of same redshift\n (bottom to top)", ncol=1)
# ax.legend(title="Lines of same redshift\n (left to right)", ncol=1)
# ax.legend(title="Lines of same snapshots (bottom to top)", ncol=3)
ax.legend(title=legend_title, ncol=1)

# ax.text(0.95, 0.05,
#        f"Colour-coded by {c_type_name} at lookbacktime = {env_grid[0, c_snp].data['lookbacktime']:.2f} Gyr",
#        transform=ax.transAxes,
#        va='bottom', ha='right')

ax.text(0.95, 0.05,
       f"Galaxy accretion fraction: {gal_grid[0, 0].data['fgal']}",
       transform=ax.transAxes,
       va='bottom', ha='right')

In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"SFRevolution_{y_type}_vs_{x_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")

### Plot molecular to stellar fraction with redshift

In [ ]:
# define which data to plot
gal_grid = gal_grid_GMS_sim
halo_grid = halo_grid_GMS_sim
env_grid = env_grid_GMS_sim

# define what to plot
x_type = 'z'
x_type_name = r'log (1 + $z$)'
x_type_unit = r''

y1_type = 'mgas'
y1_type_name = r'$M_\mathrm{gas}$'
y1_type_unit = r'$M_\odot$'

y2_type = 'mstar'
y2_type_name = r'$M_*$'
y2_type_unit = r'$M_\odot$'

c_type = 'mstar'
c_type_name = r'$M_*$'
c_type_unit = r'$M_\odot$'
c_snp = 0
# c_snp = -1
c_alpha = 0.75
which_objects = range(0, len(gal_grid), 1)

# grab plotting data
x_data = []
y_data = []
mstar_data = []
c_data = []
label_l = []
for i in tqdm(which_objects):
    x_data.append([np.log10(1. + gal.data[x_type]) for gal in gal_grid[i][:]])
    y_data.append([np.log10(gal.data[y1_type] / gal.data[y2_type]) for gal in gal_grid[i][:]])
    # y_data.append([gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([1./gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([np.log10(gal_1.data[y_type] - gal_0.data[y_type]) for gal_0, gal_1 in zip(gal_grid[i][:-1], gal_grid[i][1:])])
    # y_data.append([halo.data[y_type] for halo in halo_grid[i][:]])
    # if gal_grid[i][c_snp].data['mgas'] == 0 and halo_grid[i][c_snp].data['mtot'] != 0:
    #     _c_data_i = [np.nan]
    # else:
    #     _c_data_i = [halo_grid[i][c_snp].data['mtot'] / gal_grid[i][c_snp].data['mgas']]
    mstar_data.append([np.log10(gal.data['mstar']) for gal in gal_grid[i][:]])
    c_data.append([gal_grid[i][c_snp].data[c_type]])
    # c_data.append(_c_data_i)

# normalise the colour data to make colour the data according to it (e.g. total halo mass mtot)
c_data_range = (np.nanmin(c_data), np.nanmax(c_data))
cmap = mpl.cm.magma_r
norm = mpl.colors.LogNorm(vmin=c_data_range[0], vmax=c_data_range[1])
# norm = mpl.colors.BoundaryNorm(boundaries=np.logspace(np.log10(c_data_range[0]), np.log10(c_data_range[1]), 7), ncolors=cmap.N)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


# grab all redshifts
z_list = [gal.data['z'] for gal in gal_grid[0][:]]


In [ ]:
# initialise plot
fig, ax = plt.subplots(figsize=(9.5, 6), constrained_layout=True)

# get redshift and lookbacktime min and max
z_start = env_grid[0, 0].data['z']
z_end = env_grid[0, -1].data['z']
lbt_start = env_grid[0, 0].data['lookbacktime']
lbt_end = env_grid[0, -1].data['lookbacktime']

# # plot GMS Leslie+2020
# xlin = np.linspace(x_data[0][0], x_data[-1][0], 500)
# # ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_start, log=True), ls='-.', marker='', color='xkcd:turquoise',
# #         # label=f"{lbt_start:.2f} Gyr ago (z={z_start:.1f})"
# #         )
# ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_end, log=True), ls=':', marker='', color='xkcd:turquoise',
#         # label=f"{lbt_end:.2f} Gyr ago (z={z_end:.1f})"
#         )


# additional DASHED LINES for highlighting time evolution

# # plot lines at same redshift
# redshifts_plot_l = [2., 1.5, 1., 0.5, 0.]
# # redshifts_plot_l = np.append(np.linspace(1.99, 1.74, 6), 0.0001)
# # redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
# dl_data = redshifts_plot_l
# dl_plot_l = []
# dl_label_l = []
# for redshift in redshifts_plot_l:
#     # z_i = sgm.find_nearest_index(z_list, redshift)
#     z_i = sgm.find_nearest_index(z_list[1:], redshift)
#     z_line_x = [gal_x[z_i] for gal_x in x_data]
#     z_line_y = [gal_y[z_i] for gal_y in y_data]
#     dl_plot_l.append([z_line_x, z_line_y])
#     dl_label_l.append(f"z={redshift:.2f}")
# legend_title = "Lines of same redshift"


# # plot lines of same snapshots
# # snp_plot_l = list(range(1, 10, 1)) + list(range(10, 100, 10)) + list(range(100, 1000, 100))
# snp_plot_l = list(range(0, len(x_data[0]) + 1, 500))
# dl_data = snp_plot_l
# dl_plot_l = []
# dl_label_l = []
# for snp in snp_plot_l:
#     z_line_x = [gal_x[snp] for gal_x in x_data]
#     z_line_y = [gal_y[snp] for gal_y in y_data]
#     dl_plot_l.append([z_line_x, z_line_y])
#     dl_label_l.append(f"snap {snp}")
# legend_title = "Lines of same snapshots"


# # dashed-lines colour map
# dl_data_range = (np.nanmin(dl_data), np.nanmax(dl_data))
# dl_cmap = mpl.cm.winter_r
# dl_norm = mpl.colors.Normalize(vmin=dl_data_range[0], vmax=dl_data_range[1])
# dl_mapper = mpl.cm.ScalarMappable(norm=dl_norm, cmap=dl_cmap)
#
# for dl, dl_plot, dl_label in zip(dl_data, dl_plot_l, dl_label_l):
#     ax.plot(dl_plot[0], dl_plot[1], ls='--', color=dl_mapper.to_rgba(dl), alpha=0.5, label=dl_label)


# plot actual values
for i, (x, y, c) in enumerate(zip(x_data, y_data, c_data)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax.plot(x, y, c=c_mapped, ls='-')
    # ax.plot(x[-1], y[-1], c=c_mapped, marker='o')


# add colour bar
fig.colorbar(mapper, ax=ax, label=f"{c_type_name} [{c_type_unit}] at z = {env_grid[0, c_snp].data['z']:.2f}")


# # plot one quantity like SFR79
# for i, c, label in zip(which_objects, c_data, label_l):
#     c_mapped = mapper.to_rgba(c)
#     c_mapped[:, -1] = c_alpha
#     ax[1].plot(c_data[i], SFR79_grid_xCGsim[i, -1], c=c_mapped, marker='o', label=label)


# additional fig and axes config
fig.suptitle(f'Tracing {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)


ax.set_xlabel(f"{x_type_name}")
ax.set_ylabel(f"log ({y1_type_name} / {y2_type_name})")


# this is not good, only necessary b/c of exceptionally high values
# ax.set_xlim(4.5, 16)
# ax.set_ylim(3.8, 12.3)

# ax.set_yscale('log')



# ax.legend(title="Leslie+2020 Galaxy Main Sequence:")
# ax.legend(title="Lines of same redshift\n (bottom to top)", ncol=1)
# ax.legend(title="Lines of same redshift\n (left to right)", ncol=1)
# ax.legend(title="Lines of same snapshots (bottom to top)", ncol=3)

# ax.legend(title=legend_title, ncol=1)

# ax.text(0.95, 0.05,
#        f"Colour-coded by {c_type_name} at lookbacktime = {env_grid[0, c_snp].data['lookbacktime']:.2f} Gyr",
#        transform=ax.transAxes,
#        va='bottom', ha='right')

ax.text(0.95, 0.05,
        f"Galaxy accretion fraction: {gal_grid[0, 0].data['fgal']}",
        transform=ax.transAxes,
        va='bottom', ha='right')

In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"{x_type}_evolution_of_{y1_type}_over_{y2_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")

### Average $\mu_\mathrm{molgas}$ with redshift

In [ ]:
# initialise plot
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(9.5, 9.5), sharex='all',
                       gridspec_kw={'height_ratios': [1, 0.2, 0.2], 'hspace': 0},
                       # constrained_layout=True)
                       tight_layout=True)

# get redshift and lookbacktime min and max
z_start = env_grid[0, 0].data['z']
z_end = env_grid[0, -1].data['z']
lbt_start = env_grid[0, 0].data['lookbacktime']
lbt_end = env_grid[0, -1].data['lookbacktime']

# # plot GMS Leslie+2020
# xlin = np.linspace(x_data[0][0], x_data[-1][0], 500)
# # ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_start, log=True), ls='-.', marker='', color='xkcd:turquoise',
# #         # label=f"{lbt_start:.2f} Gyr ago (z={z_start:.1f})"
# #         )
# ax.plot(xlin, sgm.GMS_Leslie2020(xlin, z=z_end, log=True), ls=':', marker='', color='xkcd:turquoise',
#         # label=f"{lbt_end:.2f} Gyr ago (z={z_end:.1f})"
#         )


# additional DASHED LINES for highlighting time evolution

# # plot lines at same redshift
# redshifts_plot_l = [2., 1.5, 1., 0.5, 0.]
# # redshifts_plot_l = np.append(np.linspace(1.99, 1.74, 6), 0.0001)
# # redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
# dl_data = redshifts_plot_l
# dl_plot_l = []
# dl_label_l = []
# for redshift in redshifts_plot_l:
#     # z_i = sgm.find_nearest_index(z_list, redshift)
#     z_i = sgm.find_nearest_index(z_list[1:], redshift)
#     z_line_x = [gal_x[z_i] for gal_x in x_data]
#     z_line_y = [gal_y[z_i] for gal_y in y_data]
#     dl_plot_l.append([z_line_x, z_line_y])
#     dl_label_l.append(f"z={redshift:.2f}")
# legend_title = "Lines of same redshift"


# # plot lines of same snapshots
# # snp_plot_l = list(range(1, 10, 1)) + list(range(10, 100, 10)) + list(range(100, 1000, 100))
# snp_plot_l = list(range(0, len(x_data[0]) + 1, 500))
# dl_data = snp_plot_l
# dl_plot_l = []
# dl_label_l = []
# for snp in snp_plot_l:
#     z_line_x = [gal_x[snp] for gal_x in x_data]
#     z_line_y = [gal_y[snp] for gal_y in y_data]
#     dl_plot_l.append([z_line_x, z_line_y])
#     dl_label_l.append(f"snap {snp}")
# legend_title = "Lines of same snapshots"


# # dashed-lines colour map
# dl_data_range = (np.nanmin(dl_data), np.nanmax(dl_data))
# dl_cmap = mpl.cm.winter_r
# dl_norm = mpl.colors.Normalize(vmin=dl_data_range[0], vmax=dl_data_range[1])
# dl_mapper = mpl.cm.ScalarMappable(norm=dl_norm, cmap=dl_cmap)
#
# for dl, dl_plot, dl_label in zip(dl_data, dl_plot_l, dl_label_l):
#     ax.plot(dl_plot[0], dl_plot[1], ls='--', color=dl_mapper.to_rgba(dl), alpha=0.5, label=dl_label)


# plot actual average values (for all)
x_data_cumul = np.zeros_like(x_data[0])
y_data_cumul = np.zeros_like(y_data[0])
c_data_cumul = 0
count = 0
for i, (x, y, c) in enumerate(zip(x_data, y_data, c_data)):
    x_data_cumul += x
    y_data_cumul += y
    c_data_cumul += c[0]
    count += 1
x_data_cumul /= count
y_data_cumul /= count
c_data_cumul /= count
ax[0].plot(x_data_cumul, y_data_cumul, c='xkcd:medium gray', label=r"mean $\mu_\mathrm{molgas}$ for all $M_*$")



# plot actual average values for log_mstar>10
x_data_a = np.array(x_data)
x_mean = np.full_like(x_data_a[0], np.nan)
x_median = np.full_like(x_data_a[0], np.nan)
y_data_a = np.array(y_data)
y_mean = np.full_like(y_data_a[0], np.nan)
y_median = np.full_like(y_data_a[0], np.nan)
mstar_data_a = np.array(mstar_data)
mstar_mean = np.full_like(mstar_data_a[0], np.nan)
mstar_median = np.full_like(mstar_data_a[0], np.nan)
mstar_gt = 10.
n_mstar_gt = np.full_like(mstar_data_a[0], np.nan)
for i, (x_t, y_t, mstar_t) in enumerate(zip(x_data_a.T, y_data_a.T, mstar_data_a.T)):
    criterion = mstar_t>mstar_gt
    x_mean[i] = np.mean(x_t[criterion])
    x_median[i] = np.median(x_t[criterion])
    y_mean[i] = np.mean(y_t[criterion])
    y_median[i] = np.median(y_t[criterion])
    mstar_mean[i] = np.mean(mstar_t[criterion])
    mstar_median[i] = np.median(mstar_t[criterion])
    n_mstar_gt[i] = np.sum(criterion)

ax[0].plot(x_median, y_median, c='xkcd:orange', label=r"median $\mu_\mathrm{molgas}$ for $M_* > 10^{10} M_\odot$")
ax[0].plot(x_mean, y_mean, c='xkcd:red',  label=r"mean $\mu_\mathrm{molgas}$ for $M_* > 10^{10} M_\odot$")


# plot number of galaxies used for averages
ax[1].plot(x_mean, n_mstar_gt, c='xkcd:orange')
ax[1].plot(x_mean, n_mstar_gt, c='xkcd:red', ls=':')


# plot number of galaxies used for averages
ax[2].plot(x_median, mstar_median, c='xkcd:orange')
ax[2].plot(x_mean, mstar_mean, c='xkcd:red')


# # plot one quantity like SFR79
# for i, c, label in zip(which_objects, c_data, label_l):
#     c_mapped = mapper.to_rgba(c)
#     c_mapped[:, -1] = c_alpha
#     ax[1].plot(c_data[i], SFR79_grid_xCGsim[i, -1], c=c_mapped, marker='o', label=label)


# additional fig and axes config
fig.suptitle(f'Tracing {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)


ax[0].set_ylabel(f"log ({y1_type_name} / {y2_type_name})")
ax[1].set_ylabel(r"$N_\mathrm{gal}$")
ax[2].set_ylabel(r"$\overline{M}_*$ [$M_\odot$]")
ax[2].set_xlabel(f"{x_type_name}")


# this is not good, only necessary b/c of exceptionally high values
# ax.set_xlim(4.5, 16)
# ax.set_ylim(3.8, 12.3)

# ax.set_yscale('log')



# ax.legend(title="Leslie+2020 Galaxy Main Sequence:")
# ax.legend(title="Lines of same redshift\n (bottom to top)", ncol=1)
# ax.legend(title="Lines of same redshift\n (left to right)", ncol=1)
# ax.legend(title="Lines of same snapshots (bottom to top)", ncol=3)

# ax.legend(title=legend_title, ncol=1)

ax[0].legend()

# ax.text(0.95, 0.05,
#        f"Colour-coded by {c_type_name} at lookbacktime = {env_grid[0, c_snp].data['lookbacktime']:.2f} Gyr",
#        transform=ax.transAxes,
#        va='bottom', ha='right')

ax[0].text(0.95, 0.05,
        f"Galaxy accretion fraction: {gal_grid[0, 0].data['fgal']}",
        transform=ax[0].transAxes,
        va='bottom', ha='right')

In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"{x_type}_evolution_of_average_{y1_type}_over_{y2_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")

### Plot ΔSFR offset/residuals of simulated galaxies from Leslie GMS over different redshifts

In [ ]:

# compute ΔSFR values
delta_sfr = []
for x_gal, y_gal in tqdm(zip(x_data, y_data), total=len(x_data)):
    delta_sfr.append([y - sgm.GMS_Leslie2020(x, z, True) for x, y, z in zip(x_gal, y_gal, z_list)])  # also take z list


In [ ]:

# initialise plot
fig, ax = plt.subplots(figsize=(9.5, 6), constrained_layout=True)

# plot lines at same redshift
redshifts_plot_l = [2, 1.5, 1., 0.5, 0.]
# redshifts_plot_l = [6., 5., 4., 3., 2., 1., 0.]
for redshift in redshifts_plot_l:
    z_i = sgm.find_nearest_index(z_list, redshift)
    z_line_x = [gal_x[z_i] for gal_x in x_data]
    z_line_y = [gal_y[z_i] for gal_y in delta_sfr]
    ax.plot(z_line_x, z_line_y, ls='--', color='xkcd:magenta', alpha=0.5)


# plot actual values
for i, (x, y, c) in enumerate(zip(x_data, delta_sfr, c_data)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax.plot(x, y, c=c_mapped, ls='-')

# add colour bar
fig.colorbar(mapper, ax=ax, label=f"{c_type_name} [{c_type_unit}] at z = {env_grid[0, c_snp].data['z']:.2f}")


# additional fig and axes config
fig.suptitle(f'ΔSFR for {len(which_objects)} simulated galaxies from z={z_start:.1f} to z={z_end:.1f}', fontsize=16)

ax.set_xlabel(r'log $M_*$ [$M_\odot$]')
ax.set_ylabel(r'$\Delta$ SFR [dex]')

# # this is not good, only necessary b/c of exceptionally high values
# ax.set_xlim(4.5, 20)
# ax.set_ylim(-1, 11)


In [ ]:
# save plot to disk

GMS_keyword = ''
# GMS_keyword = 'p1'
# GMS_keyword = 'm1'

fig.savefig(plot_dir / f"Delta_SFRevolution_vs_{x_type}_GMS{GMS_keyword}_sim_"
                       f"from_z{z_start}_to_z{z_end}_"
                       f"{datetime.now().strftime('%Y.%m.%d-%H.%M.%S')}.png")